In [2]:
import numpy as np
import datamapplot
import matplotlib.pyplot as plt
import umap
import torch
import pandas as pd

producer_embeddings = torch.load('../consumer-producer/producer_embeddings.pt', weights_only=False)
producer_communities = np.load('../consumer-producer/producer_communities.npy')

# Load 2D embeddings if they exist, otherwise create them
try:
    embeddings_2d = np.load('producer_embeddings_2d.npy')
except FileNotFoundError:
    # Reduce dimensionality to 2D using UMAP
    reducer = umap.UMAP(n_components=2, random_state=42)
    embeddings_2d = reducer.fit_transform(producer_embeddings)
    # Save the 2D embeddings
    np.save('producer_embeddings_2d.npy', embeddings_2d)

producer_df = pd.read_parquet('../consumer-producer/producer_profiles.parquet')
producer_df['bsky_url'] = producer_df['did'].apply(lambda x: f"https://bsky.app/profile/{x}")
producer_df.head()

/home/aleck/Projects/Bluesky-GraphRec/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,producer_idx,did,handle,display_name,description,followers,following,posts,joined,error,bsky_url
0,0,did:plc:nywb5oene54cllowkfwouxzz,chadloder.bsky.social,Chad Loder,"Community activist, cybersecurity expert, citi...",54792.0,1377.0,10515.0,2023-04-30T18:10:33.161Z,None,https://bsky.app/profile/did:plc:nywb5oene54cl...
1,1,did:plc:mnfnfpykrohxbck6av3f7los,blkmatters3000.bsky.social,Michael E Hopson,Organizer. Abolitionist \nHe/him,134.0,23.0,5.0,2023-05-01T22:40:48.536Z,None,https://bsky.app/profile/did:plc:mnfnfpykrohxb...
2,2,did:plc:kfdf3ncmu3ekd3yrorivypai,lolennui.bsky.social,Amy Ash,onion lady \n\nhttps://lolennui.com,52062.0,819.0,3329.0,2023-04-27T15:52:02.340Z,None,https://bsky.app/profile/did:plc:kfdf3ncmu3ekd...
3,3,did:plc:rwbe4e7d7o3fwwcqkoyjvp4v,rui.bsky.social,Rui,None,5196.0,43075.0,29.0,2023-04-11T18:05:18.291Z,None,https://bsky.app/profile/did:plc:rwbe4e7d7o3fw...
4,4,did:plc:krt7ulietkhjowpctmv2iphb,paulio.bsky.social,Paulio 🥚,None,1730.0,956.0,1138.0,2023-05-02T02:45:41.097Z,None,https://bsky.app/profile/did:plc:krt7ulietkhjo...


In [3]:
producer_communities = producer_communities.astype(str)

In [4]:
producer_communities.shape, producer_communities

((37192,), array(['49', '27', '24', ..., '14', '93', '87'], dtype='<U11'))

In [5]:
np.unique(producer_communities)

array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58',
       '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68',
       '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98',
       '99'], dtype='<U11')

In [6]:
# Create samples for each community
import random
import json

# Create a dictionary mapping community to producer info
community_samples = {}
n_samples = 10  # number of samples per community

# Add community as a column to producer_df for easier filtering
producer_df['community'] = producer_communities

for community in np.unique(producer_communities):
    # Filter producers for this community
    community_producers = producer_df[producer_df['community'] == community]
    
    # Sample n_samples or all if less than n_samples
    sample_size = min(n_samples, len(community_producers))
    sampled_producers = community_producers.sample(n=sample_size, random_state=42)
    
    # Store relevant information
    community_samples[community] = {
        'display_names': sampled_producers['display_name'].tolist(),
        'handles': sampled_producers['handle'].tolist(),
        'descriptions': sampled_producers['description'].fillna('').tolist()
    }

# Save the samples for later use
with open('community_samples.json', 'w') as f:
    json.dump(community_samples, f, indent=2)

# Now we can proceed with the Gemma code...

In [24]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

<bos>Write me a poem about Machine Learning.

Machines, with eyes that scan the land,
Collect data, a treasure in the sand.
Algorithms dance, a symphony of code,
Unveiling patterns, a story to be told.

From medical scans to weather's unpredictable turn,
Machine learning's reach is ever unbound.
It learns from mistakes, a constant evolution,
A never-ending quest for knowledge and evolution.

With each iteration, a new insight is found,
A tapestry of data, a story profound.
From mundane tasks to grandest of dreams,
Machine learning's impact is ever apparent.

But with power comes responsibility,
A need for transparency, a moral responsibility.
Bias and discrimination, a cautionary tale,
Must be addressed with care and skill.

So let us embrace this technology's might,
To solve problems, to illuminate the night.
But let us also remember, with every line of code,
The human spirit, forever to be adored.<eos>


In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2b-it-bnb-4bit")
model = AutoModelForCausalLM.from_pretrained("unsloth/gemma-2b-it-bnb-4bit")

# Load the community samples
with open('community_samples.json', 'r') as f:
    community_samples = json.load(f)

def generate_community_description(profiles):
    # Create a prompt using the instruct format with an example
    prompt = """
I will show you several social media profiles from the same community. Please provide a short description (1-3 words) that best describes this community. Return ONLY the description, no other text, no other words of any kind.

Example:
{
Profiles:
- CryptoTrader (@trader123)
  Description: Full-time crypto trader. DeFi enthusiast. NFT collector. GM!
- Alice (@alicecrypto)
  Description: Building web3 infrastructure. ETH maxi. Love DeFi and DAOs.
- Bob (@bobchain)
  Description: Blockchain developer. Smart contract auditor. Crypto since 2015.

Your response:
defi & crypto
}

Profiles:
"""
    
    # Add the actual profiles
    for i in range(len(profiles['display_names'])):
        display_name = profiles['display_names'][i]
        handle = profiles['handles'][i]
        description = profiles['descriptions'][i][:200]  # Truncate long descriptions
        prompt += f"- {display_name} (@{handle})\n"
        prompt += f"  Description: {description}\n\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
    )
    
    response = tokenizer.decode(outputs[0])
    print("RESPONSE", response)
    return response

# Generate descriptions for each community
community_descriptions = {}
for community, profiles in community_samples.items():
    print(f"Generating description for community {community}...")
    description = generate_community_description(profiles)
    community_descriptions[community] = description
    print(f"Community {community}: {description}\n")


# Save the descriptions
with open('community_descriptions.json', 'w') as f:
    json.dump(community_descriptions, f, indent=2)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Generating description for community 0...


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [82]:
hover_text_template = """
<div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;">
    <p style="font-weight: 600; font-size: 14px; margin: 0 0 2px 0;">{hover_text}</p>
    <p style="font-size: 14px; color: #666; margin: 0 0 8px 0;">@{handle}</p>
    <p style="font-size: 14px; color: #4A4A4A; margin: 0 0 8px 0;">{description}</p>
    <div style="display: flex; gap: 16px; font-size: 13px; color: #666;">
        <span><b>{followers}</b> followers</span>
        <span><b>{following}</b> following</span>
        <span><b>{posts}</b> posts</span>
    </div>
</div>
"""

# Create the plot
plot = datamapplot.create_interactive_plot(
    embeddings_2d, 
    producer_communities,
    hover_text=producer_df['display_name'].to_list(),
    extra_point_data=producer_df[['handle','description', 'followers', 'following', 'bsky_url', 'posts']].fillna(''),
    hover_text_html_template=hover_text_template,
    on_click="window.open(hoverData.bsky_url[index], '_blank')",
    enable_search=True,
    search_field="description"
)

# Print some basic statistics about the embeddings
print(f"Original embedding shape: {producer_embeddings.shape}")
print(f"2D embedding shape: {embeddings_2d.shape}")
print(f"Number of posts: {len(producer_embeddings)}")
plot.save('producer_embeddings.html')

Original embedding shape: (37192, 64)
2D embedding shape: (37192, 2)
Number of posts: 37192
